In [5]:

import numpy as np #importing numpy
import pandas as pd #importing pandas
import tensorflow as tf #importing tensorflow
from keras.layers import Input, Dense, BatchNormalization, Reshape, Conv2D, add, LeakyReLU #importing different types of layers from keras.layers
from keras.models import Model #importing Model from keras.models

In [7]:
import math
import time
import scipy.io as sio
import requests
from keras.callbacks import TensorBoard, Callback

In [8]:
train_mat_file = sio.loadmat("D:\SURGE2023\DATA_Htrainin.mat")
train_mat_file

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Wed Feb 14 23:48:59 2018',
 '__version__': '1.0',
 '__globals__': [],
 'HT': array([[0.4985903 , 0.44685881, 0.49161327, ..., 0.49999913, 0.49999922,
         0.49999928],
        [0.48090509, 0.51376038, 0.521272  , ..., 0.49930221, 0.49933539,
         0.49936612],
        [0.43273326, 0.58984117, 0.52738918, ..., 0.5005054 , 0.50048927,
         0.5004741 ],
        ...,
        [0.4461623 , 0.51513364, 0.49219537, ..., 0.49981382, 0.49981551,
         0.49981708],
        [0.50241332, 0.56350649, 0.49639131, ..., 0.50008163, 0.50008288,
         0.50008405],
        [0.46047434, 0.18466446, 0.55279766, ..., 0.49946052, 0.4994818 ,
         0.49950172]])}

In [9]:
len(train_mat_file)

4

In [10]:
keys = list(train_mat_file.keys())
keys

['__header__', '__version__', '__globals__', 'HT']

In [11]:
test_mat_file = sio.loadmat("D:\SURGE2023\DATA_Htestin.mat")
test_mat_file

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Thu Feb 15 15:15:44 2018',
 '__version__': '1.0',
 '__globals__': [],
 'HT': array([[0.26092366, 0.85190529, 0.60494775, ..., 0.50036418, 0.50035066,
         0.50033798],
        [0.33303705, 0.55504731, 0.52144482, ..., 0.49960132, 0.49962547,
         0.49964788],
        [0.4888583 , 0.24057908, 0.51233443, ..., 0.50040599, 0.50040003,
         0.50039445],
        ...,
        [0.48810183, 0.33112218, 0.51703999, ..., 0.50035488, 0.50034619,
         0.50033811],
        [0.55198378, 0.34518412, 0.47276785, ..., 0.49952237, 0.49954685,
         0.49956975],
        [0.49810788, 0.49417168, 0.50562409, ..., 0.49978085, 0.4997903 ,
         0.49979911]])}

In [12]:
val_mat_file = sio.loadmat("D:\SURGE2023\DATA_Hvalin.mat")
val_mat_file

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Thu Feb 15 12:25:52 2018',
 '__version__': '1.0',
 '__globals__': [],
 'HT': array([[0.52056352, 0.46953018, 0.48859902, ..., 0.49981154, 0.49982013,
         0.49982817],
        [0.62065814, 0.30577164, 0.43289641, ..., 0.49939754, 0.49933464,
         0.49927567],
        [0.50879717, 0.52795825, 0.46545486, ..., 0.49941865, 0.49946489,
         0.49950812],
        ...,
        [0.55270157, 0.0092473 , 0.4575696 , ..., 0.49937621, 0.49939926,
         0.49942082],
        [0.50073979, 0.49842314, 0.49969159, ..., 0.50002873, 0.50002719,
         0.50002578],
        [0.53566402, 0.45551388, 0.52186067, ..., 0.49834792, 0.49843517,
         0.49851627]])}

In [13]:
x_train = train_mat_file["HT"]
x_test = test_mat_file["HT"]
x_val = val_mat_file["HT"]

In [14]:
print(x_train.shape)
print(x_test.shape)
print(x_val.shape)

(100000, 2048)
(20000, 2048)
(30000, 2048)


In [15]:
x_train = x_train.astype(float)
x_test = x_test.astype(float)
x_val = x_val.astype(float)

In [16]:
height = 32
width = 32
channels = 2
total = height*width*channels  

In [17]:
residual = 2
encoded_dim = 512
initial_lr = 5e-3
lr_drop_period = 100
lr_drop_factor = 0.1



In [18]:
def network(x, residual, encoded_dim):
    def addcommonlayers(y):
        y = BatchNormalization()(y)
        y = LeakyReLU()(y)
        return y
    
    def decoded(y):
        temp = y
        y = Conv2D(8, kernel_size=(3,3),padding = 'same' )(y)
        y = addcommonlayers(y)
        
        y = Conv2D(16, kernel_size = (3,3),padding = 'same')(y)
        y = addcommonlayers(y)

        y = Conv2D(2,kernel_size = (3,3),padding = 'same')(y)
        y = BatchNormalization()(y)

        y = add([temp,y])
        y = LeakyReLU()(y)

        return y
    
    x = Conv2D(2,(3,3),padding = 'same')(x)
    x = addcommonlayers(x)

    x = Reshape((total,))(x)
    encoded = Dense(encoded_dim, activation = 'linear')(x)

    x = Dense(total,activation='linear')(x)
    x = Reshape((height,width,channels))(x)

    for i in range(residual):
        x = decoded(x)

    x = Conv2D(2,(3,3),activation = 'sigmoid',padding = 'same')(x)

    return x





In [19]:
image_tensor = Input(shape=(height,width,channels))
network_output = network(image_tensor, residual, encoded_dim)
autoencoder = Model(inputs=[image_tensor], outputs=[network_output])
autoencoder.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=initial_lr), loss='mse')
print(autoencoder.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 2)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 2)    38          ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 2)   8           ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 leaky_re_lu (LeakyReLU)        (None, 32, 32, 2)    0           ['batch_normalization[0][0]']

In [20]:
x_train = np.reshape(x_train, (len(x_train), channels, height, width))
x_train = np.transpose(x_train, (0,2,3,1)) # remove this line if using `channels_first` image data format 
x_val = np.reshape(x_val, (len(x_val), channels, height, width))
x_val = np.transpose(x_val, (0,2,3,1)) # remove this line if using `channels_first` image data format
x_test = np.reshape(x_test, (len(x_test), channels, height, width))
x_test = np.transpose(x_test, (0,2,3,1)) # remove this line if using `channels_first` i

In [21]:
class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses_train = []
        self.losses_val = []

    def on_epoch_end(self, epoch, logs={}):
        self.losses_train.append(logs.get('loss'))
        
    def on_epoch_end(self, epoch, logs={}):
        self.losses_val.append(logs.get('val_loss'))

    def scheduler(epoch, lr):
        if lr_drop_period == np.Inf or np.mod(epoch,lr_drop_period) != 0:
            return lr
        else:
            return lr * tf.math.exp(-lr_drop_factor)

In [22]:
history = LossHistory()
file = 'CsiNet_'+'_dim'+str(encoded_dim)+time.strftime('_%m_%d')
path = 'result/TensorBoard_%s' %file

In [23]:
earlyStopping = tf.keras.callbacks.EarlyStopping(patience=200,restore_best_weights=True)

In [24]:
autoencoder.fit(x_train, x_train,
                epochs=10,
                batch_size=200,
                shuffle=True,
                validation_data=(x_val, x_val),
                callbacks=[history,
                           TensorBoard(log_dir = path), earlyStopping])

Epoch 1/10
500/500 [==============================] - 720s 1s/step - loss: 7.1282e-04 - val_loss: 4.3538e-04
Epoch 2/10
500/500 [==============================] - 449s 897ms/step - loss: 2.4128e-04 - val_loss: 0.0340
Epoch 3/10
500/500 [==============================] - 1419s 3s/step - loss: 1.6957e-04 - val_loss: 3.0796e-04
Epoch 4/10
500/500 [==============================] - 442s 883ms/step - loss: 1.3455e-04 - val_loss: 0.0541
Epoch 5/10
500/500 [==============================] - 440s 879ms/step - loss: 1.1104e-04 - val_loss: 7.5473e-04
Epoch 6/10
500/500 [==============================] - 439s 878ms/step - loss: 1.0605e-04 - val_loss: 0.0418
Epoch 7/10
500/500 [==============================] - 671s 1s/step - loss: 9.4285e-05 - val_loss: 0.0050
Epoch 8/10
500/500 [==============================] - 584s 1s/step - loss: 8.5542e-05 - val_loss: 2.7708e-04
Epoch 9/10
500/500 [==============================] - 442s 885ms/step - loss: 7.4290e-05 - val_loss: 1.4577e-04
Epoch 10/10
500/500

In [25]:
filename = 'result/trainloss_%s.csv'%file
loss_history = np.array(history.losses_train)
np.savetxt(filename, loss_history, delimiter=",")

In [26]:
filename = 'result/valloss_%s.csv'%file
loss_history = np.array(history.losses_val)
np.savetxt(filename, loss_history, delimiter=",")

In [27]:
tStart = time.time()
x_hat = autoencoder.predict(x_test)
tEnd = time.time()
print ("It cost %f sec" % ((tEnd - tStart)/x_test.shape[0]))

625/625 [==============================] - 26s 41ms/step
It cost 0.001605 sec


In [38]:
x_hat

array([[[[0.32546124, 0.27011594],
         [0.8304058 , 0.8114788 ],
         [0.5705308 , 0.6043475 ],
         ...,
         [0.50680053, 0.5057155 ],
         [0.5085773 , 0.50607395],
         [0.50178695, 0.49627075]],

        [[0.5940764 , 0.5842896 ],
         [0.44206798, 0.47386715],
         [0.5060197 , 0.49213055],
         ...,
         [0.4991824 , 0.50473976],
         [0.49007744, 0.4996711 ],
         [0.50548   , 0.49842408]],

        [[0.6579405 , 0.5206724 ],
         [0.26405266, 0.40053877],
         [0.420524  , 0.47141343],
         ...,
         [0.49564248, 0.5029201 ],
         [0.4911952 , 0.50171936],
         [0.5032571 , 0.49525535]],

        ...,

        [[0.48279542, 0.4737401 ],
         [0.49904382, 0.5087365 ],
         [0.49327132, 0.50682914],
         ...,
         [0.4988521 , 0.4981403 ],
         [0.4982949 , 0.5034638 ],
         [0.50422394, 0.49965367]],

        [[0.51362365, 0.4802076 ],
         [0.4936279 , 0.51500064],
         [0.

In [45]:
def NMSE(x_i,x_hat):
    x_i_real = np.reshape(x_i[:,:,:,0],(len(x_i),-1))
    x_i_img  = np.reshape(x_i[:,:,:,1],(len(x_i),-1))
    x_i_C = x_i_real-0.5 + 1j*(x_i_img-0.5)

    x_hat_real = np.reshape(x_hat[:,:,:,0],(len(x_i),-1))
    x_hat_img  = np.reshape(x_hat[:,:,:,1],(len(x_i),-1))
    x_hat_C = x_hat_real-0.5 + 1j*(x_hat_img-0.5)

    mse = np.mean(np.abs(x_i_C-x_hat_C)**2, axis=1)
    power = np.sum(np.abs(x_i_C)**2, axis=1)
    NMSE = mse/power
    NMSE = 10*np.log10(NMSE)
    NMSE = np.sum(NMSE)
    return NMSE


In [46]:


print("NMSE is ", NMSE(x_test,x_hat)/(x_text.shape[0]))
filename = "result/decoded_%s.csv"%file
x_hat1 = np.reshape(x_hat, (len(x_hat), -1))
np.savetxt(filename, x_hat1, delimiter=",")

NMSE is  -756404.004370677


In [47]:
# Serialize model to JSON
model_json = autoencoder.to_json()
outfile = "result/model_%s.json"%file
with open(outfile, "w") as json_file:
    json_file.write(model_json)

# Serialize weights to HDF5
outfile = "result/model_%s.h5"%file
autoencoder.save_weights(outfile)

# Save model in tensorflow format
autoencoder.save("tensorflow/model_%s"%file)

INFO:tensorflow:Assets written to: tensorflow/model_CsiNet__dim512_05_26\assets


INFO:tensorflow:Assets written to: tensorflow/model_CsiNet__dim512_05_26\assets
